### Options Pricer

Saurav Luthra, Dec 2021

My goal is to calculate the price of options along a certain stock's option chain using the Black-Scholes Model, and eventually scrape real-time market data to compare real bid/ask prices to calculated prices. The stock whose options chain is modeled will be user selected, as well as the expiry date for the option chain.

I'm hoping this options pricer project will lead me to a more in-depth understanding of options pricing and volatility, as well as the technical tools needed to create an algorithmic trading strategy from scratch. This project will not account for transaction costs and will not be able to execute trades on its own (due to some external restrictions) however I hope to implement such features in the future.

In [9]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
import os

ImportError: dlopen(/Users/sauravluthra/Desktop/Options_Pricer/options_pricer/Options_Pricer/scipy/_lib/_ccallback_c.cpython-39-darwin.so, 2): no suitable image found.  Did find:
	/Users/sauravluthra/Desktop/Options_Pricer/options_pricer/Options_Pricer/scipy/_lib/_ccallback_c.cpython-39-darwin.so: mach-o, but wrong architecture
	/Users/sauravluthra/Desktop/Options_Pricer/options_pricer/Options_Pricer/scipy/_lib/_ccallback_c.cpython-39-darwin.so: mach-o, but wrong architecture

In [10]:
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [14]:
stock = 'AAPL'
expiry = '12-18-2023'
strike_price = 200

today = datetime.now()
one_year_ago = today.replace(year=today.year-1)

print("Today, One Yr Ago:", today, one_year_ago)

Today, One Yr Ago: 2021-12-31 04:52:27.406900 2020-12-31 04:52:27.406900


In [22]:
os.environ["IEX_API_KEY"] = "pk_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
df = web.DataReader('gs', 'iex-tops')
df[:10]

TypeError: ignored

In [20]:
df = web.DataReader(stock, 'iex', one_year_ago.date(), today.date())

df = df.sort_values(by="Date")
df = df.dropna()
df = df.assign(close_day_before=df.Close.shift(1))
df['returns'] = ((df.Close - df.close_day_before)/df.close_day_before)

sigma = np.sqrt(252) * df['returns'].std()
uty = (web.DataReader(
    "^TNX", 'yahoo', today.replace(day=today.day-1), today)['Close'].iloc[-1])/100
lcp = df['Close'].iloc[-1]
t = (datetime.strptime(expiry, "%m-%d-%Y") - datetime.utcnow()).days / 365

print('The Option Price is: ', bs_call(lcp, strike_price, t, uty, sigma))

RemoteDataError: ignored

In [ ]:
def call_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm.cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm.cdf(d2(S, K, T, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

print("Implied Volatility: " +
      str(100 * call_implied_volatility(bs_call(lcp, strike_price, t, uty, sigma,), lcp, strike_price, t, uty,)) + " %")

In [11]:
def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
    
def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))